In [291]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [292]:
data = pd.read_csv('./data/properatti_clean_model.csv')
data

,Unnamed: 0,property_type,place_name,state_name,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,garage,lavadero,...,sum,quincho,estrenar,min_subte,subtes_menor_500m,min_locales_bailables,locales_bailables_menor_500m,min_comisarias,comisarias_menor_500m,min_cuarteles
0,0,PH,Mataderos,capital federal,62000.0,55.000000,40.000000,2.000000,0,1,...,0,0,0,4728.393676,0.000000,1666.691008,0.000000,854.731083,0.000000,1338.362957
1,2,apartment,Mataderos,capital federal,72000.0,55.000000,55.000000,2.000000,0,1,...,0,0,0,6073.153602,0.000000,951.482447,0.000000,1329.365030,0.000000,1349.185580
2,3,PH,Liniers,capital federal,95000.0,120.857451,91.601224,3.000000,0,0,...,0,0,0,5305.630900,0.000000,536.546481,0.000000,1342.826496,0.000000,1342.048882
3,7,apartment,Belgrano,capital federal,138000.0,45.000000,40.000000,2.852152,0,1,...,0,0,1,609.743234,0.000000,552.466358,0.000000,305.293658,1.000000,1330.713856
4,8,apartment,Belgrano,capital federal,195000.0,65.000000,60.000000,2.852152,0,1,...,0,0,1,609.743234,0.000000,552.466358,0.000000,305.293658,1.000000,1330.713856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28394,121160,apartment,Belgrano,capital federal,410000.0,157.000000,140.112501,2.852152,0,1,...,0,0,0,430.624480,1.000000,498.301841,1.000000,540.154238,0.000000,1113.380290
28395,121209,apartment,Belgrano,capital federal,410000.0,157.000000,140.112501,2.852152,0,1,...,0,0,0,430.624480,1.000000,498.301841,1.000000,540.154238,0.000000,1113.380290
28396,121215,apartment,Belgrano,capital federal,870000.0,113.000000,93.000000,2.000000,1,0,...,0,1,0,466.134341,0.608407,456.041680,0.782080,513.679193,0.501106,762.594984
28397,121217,apartment,Villa Urquiza,capital federal,131500.0,46.000000,39.000000,2.509056,1,1,...,0,0,1,637.215519,0.000000,1569.942446,0.000000,146.700180,1.000000,1088.068227


In [293]:
# Limpiando

# data.drop(columns=['price_aprox_usd','Unnamed: 0'], inplace=True)
data.drop(columns=['Unnamed: 0'], inplace=True)

#Quitamos el primer cuantile porque tiene valores en 0
# data = data[data.surface_total_in_m2 > data.surface_total_in_m2.quantile(0.01)]
data = data[data.price_aprox_usd > 0]
data = data[data.surface_covered_in_m2 > 0]
data = data[data.surface_total_in_m2 > 0]

#Nos quedamos solo con Capital Federal
data = data[data.state_name.isin(['capital federal'])]
data.drop(columns=['state_name'],inplace=True)

In [294]:
data.surface_total_in_m2.min()

8.315254097769142

Observaciones:
- Pasar el modelo como esta el dataset hoy a mí me dio un r2 de aprox 0,03 a 0,05
- Mejoró cuando se tomo una zona muy especifica como ser capital federal

In [295]:
# data.drop(columns=['min_subte','subtes_menor_500m'],inplace=True)
# data.drop(columns=['min_subte', 'min_locales_bailables'],inplace=True)

In [296]:
categorical_columns = data.columns[data.dtypes == 'object']
categorical_columns

Index(['property_type', 'place_name'], dtype='object')

In [297]:
df_dummies = pd.get_dummies(data[categorical_columns],drop_first=True)
df_dummies

,property_type_apartment,property_type_house,property_type_store,place_name_Agronomía,place_name_Almagro,place_name_Balvanera,place_name_Barracas,place_name_Barrio Norte,place_name_Belgrano,place_name_Boca,...,place_name_Villa Lugano,place_name_Villa Luro,place_name_Villa Ortuzar,place_name_Villa Pueyrredón,place_name_Villa Real,place_name_Villa Riachuelo,place_name_Villa Santa Rita,place_name_Villa Soldati,place_name_Villa Urquiza,place_name_Villa del Parque
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28394,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
28395,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
28396,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
28397,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [298]:
data.drop(columns=categorical_columns,inplace=True)

In [299]:
df = pd.concat([data,df_dummies],axis=1)
df

,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,rooms,garage,lavadero,balcon,parrilla,piscina,terraza,...,place_name_Villa Lugano,place_name_Villa Luro,place_name_Villa Ortuzar,place_name_Villa Pueyrredón,place_name_Villa Real,place_name_Villa Riachuelo,place_name_Villa Santa Rita,place_name_Villa Soldati,place_name_Villa Urquiza,place_name_Villa del Parque
0,62000.0,55.000000,40.000000,2.000000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,72000.0,55.000000,55.000000,2.000000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,95000.0,120.857451,91.601224,3.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,138000.0,45.000000,40.000000,2.852152,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,195000.0,65.000000,60.000000,2.852152,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28394,410000.0,157.000000,140.112501,2.852152,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28395,410000.0,157.000000,140.112501,2.852152,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28396,870000.0,113.000000,93.000000,2.000000,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
28397,131500.0,46.000000,39.000000,2.509056,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0


In [300]:
df.dtypes.unique()

array([dtype('float64'), dtype('int64'), dtype('uint8')], dtype=object)

### Creando interacciones

In [301]:
data.columns

Index(['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
       'rooms', 'garage', 'lavadero', 'balcon', 'parrilla', 'piscina',
       'terraza', 'patio', 'jardin', 'sum', 'quincho', 'estrenar', 'min_subte',
       'subtes_menor_500m', 'min_locales_bailables',
       'locales_bailables_menor_500m', 'min_comisarias',
       'comisarias_menor_500m', 'min_cuarteles'],
      dtype='object')

In [302]:
# df['garage_m2_cub'] = df.garage * df.surface_covered_in_m2
# df['']

Empezamos a construir el modelo

In [303]:
target_column_name = 'price_aprox_usd'

In [304]:
X = df.drop(columns=target_column_name)
y = df[target_column_name]

# Tenemos que agregar explícitamente a una constante:
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        price_aprox_usd   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.597
Method:                 Least Squares   F-statistic:                     494.9
Date:                Mon, 03 Oct 2022   Prob (F-statistic):               0.00
Time:                        22:52:06   Log-Likelihood:            -3.7576e+05
No. Observations:               28384   AIC:                         7.517e+05
Df Residuals:                   28298   BIC:                         7.524e+05
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                           -5.119e+04   1.47e+04     -3.491      0.000   -7.99e+04   -2.24e+04
surface_total_in_m2               113.9459     20.296      5.614      0.000      74.164     153.727
surface_covered_in_m2            1818.5025     30.217     60.181      0.000    1759.275    1877.730
rooms                            1.553e+04    862.487     18.001      0.000    1.38e+04    1.72e+04
garage                           1.796e+04   1806.153      9.945      0.000    1.44e+04    2.15e+04
lavadero                         1.822e+04   1816.489     10.033      0.000    1.47e+04    2.18e+04
balcon                          -4302.7542   1787.109     -2.408      0.016   -7805.572    -799.936
parrilla                         1.977e+04   2284.144      8.656      0.000    1.53e+04    2.42e+04
piscina                          6633.8133   2409.831      2.753      0.006    1910.430    1.14e+04
terraza                         -1036.6951   2130.149     -0.487      0.626   -5211.888    3138.498
patio                           -1.873e+04   2631.373     -7.118      0.000   -2.39e+04   -1.36e+04
jardin                           2.132e+04   3340.963      6.382      0.000    1.48e+04    2.79e+04
sum                               3.83e+04   2708.455     14.141      0.000     3.3e+04    4.36e+04
quincho                          8084.6806   4356.329      1.856      0.063    -453.932    1.66e+04
estrenar                        -1.177e+04   2848.718     -4.132      0.000   -1.74e+04   -6187.130
min_subte                          -0.6788      2.551     -0.266      0.790      -5.679       4.322
subtes_menor_500m               -1.101e+04   1638.225     -6.722      0.000   -1.42e+04   -7800.798
min_locales_bailables              -8.1843      3.092     -2.647      0.008     -14.244      -2.124
locales_bailables_menor_500m      -58.7408    397.978     -0.148      0.883    -838.796     721.315
min_comisarias                      2.5953      3.803      0.682      0.495      -4.859      10.050
comisarias_menor_500m           -6112.0548   1892.837     -3.229      0.001   -9822.105   -2402.004
min_cuarteles                       6.1767      2.564      2.409      0.016       1.150      11.203
property_type_apartment          3.809e+04   3864.821      9.855      0.000    3.05e+04    4.57e+04
property_type_house             -5.309e+04   5209.568    -10.190      0.000   -6.33e+04   -4.29e+04
property_type_store              7.299e+04   5699.956     12.806      0.000    6.18e+04    8.42e+04
place_name_Agronomía            -4.144e+04   2.65e+04     -1.566      0.117   -9.33e+04    1.04e+04
place_name_Almagro               3600.4371   1.46e+04      0.247      0.805   -2.49e+04    3.21e+04
place_name_Balvanera            -1.104e+04   1.48e+04     -0.745      0.456   -4.01

In [305]:
df.shape

(28384, 86)

In [306]:
# relevant_columns = model.pvalues[model.pvalues < 0.05].index[1:].to_list()
# relevant_columns

In [307]:
# X = df[['surface_total_in_m2']]
# y = df.price_aprox_usd

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 123)

# scaler = StandardScaler()
# X_train_std = scaler.fit_transform(X_train)

# model_ridge_cv = linear_model.RidgeCV(alphas= [0.3, 0.5, 1.0, 1.1, 1.15, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.3, 1.4, 1.5, 10,15,20,25,30,35], 
#                                    fit_intercept=True, normalize=False, cv=10)

# model_fit_ridge_cv = model_ridge_cv.fit(X_train_std, y_train)

# print(model_fit_ridge_cv.alpha_)

# print(model_fit_ridge_cv.best_score_)

# X_test_std = scaler.fit_transform(X_test)

# print(model_fit_ridge_cv.score(X_test_std, y_test))

In [308]:
# print(model_fit_ridge_cv.score(X_train_std, y_train))

In [309]:
# TEST WiTh ALL Columns
X = df.drop(columns=target_column_name)
y = df[target_column_name]

# TEST WITH SELECTED COLUMNS
# X = df[relevant_columns]
# y = df[target_column_name]


In [310]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

#### Ridge 

In [311]:
model_ridge_cv = linear_model.RidgeCV(alphas= np.logspace(-10,2,200), 
                                   fit_intercept=True, normalize=False, cv=10)

In [312]:
model_fit_ridge_cv = model_ridge_cv.fit(X_train_std, y_train)

In [313]:
print(model_fit_ridge_cv.alpha_)

print(model_fit_ridge_cv.best_score_)

1e-10
0.5944808458284838


In [314]:
X_test_std = scaler.fit_transform(X_test)

In [315]:
print(model_fit_ridge_cv.score(X_train_std, y_train))

0.5981265955078241


In [316]:
print(model_fit_ridge_cv.score(X_test_std, y_test))

0.566630846039819


## Lasso

In [317]:
model_lasso_cv = linear_model.LassoCV(alphas= np.logspace(-10,2,200), 
                                   fit_intercept=True, normalize=False, cv=10, max_iter=1e4)

In [318]:
model_fit_lasso_cv = model_lasso_cv.fit(X_train_std, y_train)

c:\Users\Usuario\anaconda3\envs\geopandassb\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2840103244103.3125, tolerance: 93078823680.59401
  tol, rng, random, positive)
c:\Users\Usuario\anaconda3\envs\geopandassb\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6784594151503.25, tolerance: 93078823680.59401
  tol, rng, random, positive)
c:\Users\Usuario\anaconda3\envs\geopandassb\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:527: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11061120672858.75, tolerance: 93078823680.59401
  tol, rng, random, positive)
c:\Users\Usuario\anaconda3\envs\geopandassb\lib\site-packages\sklearn\linear_model\_coordin

In [319]:
print(model_fit_lasso_cv.alpha_)

print(model_fit_lasso_cv.score(X_train_std, y_train))

1e-10
0.5981265955078241


In [320]:
print(model_fit_lasso_cv.score(X_test_std, y_test))

0.566630846039813


In [321]:
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(model_fit_lasso_cv, X, y, cv=5)
# scores


In [322]:
# scores.mean()

Da los mismos resultados que esto:

In [323]:
# from sklearn.pipeline import make_pipeline
# clf = make_pipeline(StandardScaler(), model_lasso_cv)
# scores = cross_val_score(clf, X, y, cv=20)
# scores

In [324]:
# scores.mean()

In [325]:
# from sklearn import metrics

# y_pred = model_fit_lasso_cv.predict(y_test)


# # Métricas sobre test
# estimadores = len(X_test.columns)
# observaciones = len(y_test)
# r2 = metrics.r2_score(y_test, y_pred)
# adj_r2 = 1-(1-r2)*((observaciones-1)/(observaciones-estimadores-1))
# print("R2: ", r2)
# print("Adj R2: ",adj_r2)
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


